#### Attempt at CP implementation for tensor decomposition

Here is the pseudocode we will try to for the function cp.

def cp(X,R):

initialize $A^{(n)} \in \mathbb{R}^{I_n \times R}$ for $n = 1, ... N$
    
repeat

for $n = 1, ..., N$ do
        
$V = {A^{(1)}}^T A^{(1)} * ... * {A^{(n-1)}}^T A^{(n-1)} * {A^{(n+1)}}^T A^{(n+1)} * ... * {A^{(N)}}^T A^{(N)}$

$A^{(n)} = X^{(n)} (A^{(N)} \odot ... \odot A^{(n+1)} \odot A^{(n-1)} \odot ... \odot A^{(1)}) V^{\dagger}$

normalize columns of A and store norms $\lambda$

end for

until fit ceases to improve or maximum iterations exhausted

return $\lambda, A^{(1)}, A^{(2)}, ..., A^{(N)}$

end procedure

In [32]:
import numpy as np
from scipy import linalg as LA
# import tensorly as tl

# function implementation of cp

def cp(X,R):
    
    # shape of tensor X, returns tuple of dimensions
    shp = np.shape(X)
    N = np.ndim(X)
    
    # initialize array of all A^(n)
    A = np.array()
    
    # fill in array of all A^(n) with each A^(n) being a 
    # 2D array of dimensions I_n * R and filled with ones
    
    for i in range(0,N):
        A[i] = np.ones([shp[i], R], dtype = int)
        
    # repeat updates to all A^(n) until fit ceases to 
    # improve or max iterations exhausted
    
    lmd = np.array() # lambda
    
    maxit = 100 # max iterations
    
    while (it < maxit):
        
        for j in range(0,N):
            
            # calculate V
            V = np.array(int)
            for k in range(0,j) + range(j+1, N):
                V = V * np.matmul(A[k].transpose, A[k])
                
            # calculate A
            for m in range(N-1,j-1, -1) + range(j-2, -1, -1):
                A[j] = LA.khatri_rao(A[j], A[m])
                
            A[j] = np.reshape(X, (shp[j], np.size(X)/shp[j])) * A[j] * np.pinv(V)
            
            # normalize columns
            lmd = LA.norm(A[j])
            A[j] = A[j] / lmd
        
        it += 1
        
    return lmd, A

In [33]:
# Testing

# initialize the tensor X
X = np.random.rand(5,6,7,8)

print(cp(X,5))

MemoryError: Unable to allocate 95.1 PiB for an array with shape (5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8) and data type float64